## [Meta Coding Puzzles](https://www.metacareers.com/profile/coding_puzzles/)

Below are my solutions for the "Level 1" puzzles.

### Cafeteria
Time limit: 5s

A cafeteria table consists of a row of $N$ seats, numbered from 
$1$ to $N$ from left to right. Social distancing guidelines require that every diner be seated such that $K$ seats to their left and 
$K$ seats to their right (or all the remaining seats to that side if there are fewer than $K$) remain empty.

There are currently M diners seated at the table, the $i$th of whom is in seat $S_i$. No two diners are sitting in the same seat, and the social distancing guidelines are satisfied.

Determine the maximum number of additional diners who can potentially sit at the table without social distancing guidelines being violated for any new or existing diners, assuming that the existing diners cannot move and that the additional diners will cooperate to maximize how many of them can sit down.

*Please take care to write a solution which runs within the time limit.*

**Constraints**

$ 1 \leq N \leq 10^{15} \\
1 \leq K \leq N \\
1 \leq M \leq 500{,}000 \\
M \leq N \\
1 \leq S_i \leq N$

**Sample test cases**
1. $N = 10 \\
K = 1 \\
M = 2 \\
S = [2, 6]$

&emsp;&emsp;Expected return value = 3

2. $N = 15 \\
K = 2 \\
M = 3 \\
S = [11, 6, 14]$

&emsp;&emsp;Expected return value = 1

#### Solution

The first thing I notice is that the upper bound on $N$ is quite large, so I know I'll have to make sure to not loop through $N$ any more than what is necessary to be able to run in the 5s time limit.

$M$, the number of people currently seated is below 500,000, much below the upper bound on $N$. Looking at the test cases, I also notice that $S$ is not necessarily sorted.

Here's a sketch of my approach:
1. Sort $S$.
2. Use division to determine how many diners can sit between $1$ and $S_1$.
3. Use division to determine how many diners can sit between $S_M$ and $N$.
4. Loop through the seats from $S_1$ to $S_M$ to determine how many diners can fit in between.

One thing I'm worried about is whether it's possible that I could get a different answer depending exactly how I fill the seats, whether going up or going down. I think I may be OK though because of symmetry though. For example, say there's diners at 2 and 7 with $K=1$. I could put a diner at 4 (1 open to left and 2 to right) or 5 (2 open to left and 1 to right). If the original diner is in 8 rather than 7, I can fit 2 diners in seats 4 and 6, and it doesn't matter whether I start at the left or right. If the diners are at 2 and 9, I can start at 2 going up and put diners at 4 and 6, or I can start at 9 and go down, putting diners at 7 and 5. I get different orientations but the same number of new diners. This is not a proof that I'll be OK in all situations but I can't immediately come up with a scenario counter to my intuition so I think I'm OK and I'll proceed and see if I run into any obvious issues. (Update: see paragraph below next for new approach).

Moving onto Step 2 in the above sketch. Given a diner at $S_1 \leq S_i$ for all $i$, I want to try to fit in as many diners in the seats to the left. Rather than looping through and adding diners, one by one I can just use division to determine where diners can go. I am essentially trying to place $K + 1$ blocks into these seats (1 for the diner plus $K$ seats to one side. If everyone has $K$ seats on one side of them then the distancing requirements will be satisfied). Therefore, I just need the number of whole $K + 1$ pieces I can fit in to the $S_1 - 1$ seats. This is given by integer division, which drops the remainder. Similarly, for $S_M$, I want to fit $K + 1$ pieces into the $N - S_M$ rightmost seats. 

Working through that arithmetic above, I see that I can modify Step 4 above and not worry about how exactly I fill the seats. Given diners at $S_i$ and $S_{i+1}$ I can use a similar caluclation to fit $K + 1$ pieces between the diners. The one slight difference is that with $S_1$ and $S_M$, I didn't need to worry about one of the boundaries; I could put a diner at seat $1$ or $N$ without meeting the distancing requirement on that side since there were no more seats to one side. Now I need to make sure that the diners I place meet the requirements on both sides. If I just place $K + 1$ blocks between the diners, I risk accidentally putting a diner too close to $S_i$ or $S_{i+1}$. If the $K + 1$ block has the diner with $K$ empty seats to the right, I need to make sure I remove the $K$ seats to the right of $S_i$ from consideration. Similarly, if I imagine the block are a diner with $K$ seats to the left, I can't place this block within $K$ seats of $S_{i+1}$. Either way this is a subtraction of $K$ seats, i.e. I am putting $K + 1$ blocks into a set of $S_{i+1} - S_i - 1 - K$ seats. 

Finally, let me modify the approach:
1. Sort $S$.
2. Use integer division to determine how many diners can sit between $1$ and $S_1$.
3. Use integer division to determine how many diners can sit between $S_M$ and $N$.
4. Loop through $S$, calculating available seats $S_{i+1} - S_{i} - 1$.
5. Use integer division to determine how many diners can fit in between $S_{i}$ and $S_{i+1}$.

In [29]:
from typing import List
# Write any import statements here
import numpy as np

def getMaxAdditionalDinersCount(N: int, K: int, M: int, S: List[int]) -> int:
    # Write your code here
    # want S in order from smallest to largest filled seat
    S = np.sort(S)
    
    # diners will hold how many new diners we can fit
    # first add diners that can fit in open left/right seats
    # following explanation above
    diners = 0
    diners += (S[0] - 1) // (K + 1)
    print(S[0], (S[0] - 1), K, (S[0] - 1) // (K + 1))
    diners += (N - S[-1]) // (K + 1)
    print(N, N - S[-1], K, 
          (N - S[-1]) // (K+1))
    
    # use numpy to caluclate difference between elements
    # then do integer division as described in description above
    # use numpy sum to get total of all new diners between other diners
    # and add to total, then return
    diffs = np.diff(S)
    diffs = (diffs - 1 - K) // (K + 1)
    print(np.sum(diffs))
    diners += np.sum(diffs)
    return diners

In [28]:
print(getMaxAdditionalDinersCount(10, 1, 2, [2, 6]))
print(getMaxAdditionalDinersCount(15, 2, 3, [11, 6, 14]))

2 1 1 0
10 4 1 2
1
3

6 5 2 1
15 1 2 0
0
1


**Result: 32/32 test cases solved**

### Director of Photography (Chapter 1)
Time limit: 5s

*Note: Chapter 2 is a harder version of this puzzle. The only difference is a larger constraint on $N$.* 

A photography set consists of N cells in a row, numbered from 
$1$ to $N$ in order, and can be represented by a string $C$ of length $N$. Each cell $i$ is one of the following types (indicated by $C_i$, the $i$th character of C):
* If $C_i$ = “P”, it is allowed to contain a photographer
* If $C_i$ = “A”, it is allowed to contain an actor
* If $C_i$ = “B”, it is allowed to contain a backdrop
* If $C_i$ = “.”, it must be left empty

A *photograph* consists of a photographer, an actor, and a backdrop, such that each of them is placed in a valid cell, and such that the actor is between the photographer and the backdrop. Such a photograph is considered artistic if the distance between the photographer and the actor is between $X$ and $Y$ cells (inclusive), and the distance between the actor and the backdrop is also between $X$ and $Y$ cells (inclusive). The distance between cells $i$ and $j$ is $∣i−j∣$ (the absolute value of the difference between their indices).

Determine the number of different artistic photographs which could potentially be taken at the set. Two photographs are considered different if they involve a different photographer cell, actor cell, and/or backdrop cell.

**Constraints**

$1 \leq N \leq 200 \\
1 \leq X \leq Y \leq N$

**Sample test cases**
1. $N = 5 \\
C = APABA \\
X = 1 \\
Y = 2$

&emsp;&emsp;Expected return value = 1

2. $N = 5 \\
C = APABA \\
X = 2 \\
Y = 3$

&emsp;&emsp;Expected return value = 0

3. $N = 8 \\
C = \, .PBAAP.B \\
X = 1 \\
Y = 3$

&emsp;&emsp;Expected return value = 3

#### Solution

I believe to do this I will have to check each possible placement of $P, A, B$ to see if it is a photograph and if so, whether it is artistic. To do this, I will have 3 nested loops, over the allowed locations of each element.

A sketch of my solution is as follows:
1. Get possible locations of each element.
2. Loop through all possible combinations of 3 elements.
    1. For each combination, check if it satisfies requirements for a photograph.
        1. For each photograph, check if it satisfies requirements for an artistic photograph
3. Return number of artistic photographs.

In [37]:
# helper function for checking whether a given orientation is a(n artistic) photograph
# to do this, I just need to index of each element and the X and Y values
# first I'll check for valid photograph, returning false if it is not valid
# then I'll check for artistic photograph, returning false if it is not artistic
# finally, if it passes all requirements, return true
def checkArtisticPhotograph(P: int, A: int, B: int, X: int, Y: int) -> bool:
    # check orientation, A needs to be in between P and B
    if P < B and (A > B or A < P):
        return False
    elif B < P and (A < B or A > P):
        return False
    else:
        # orientation valid, check for distances
        if abs(A - P) < X or abs(A - P) > Y:
            return False
        elif abs(A - B) < X or abs(A - B) > Y:
            return False
    # passed all checks
    return True

def getArtisticPhotographCount(N: int, C: str, X: int, Y: int) -> int:
    # list comprehension gets array of each allowed index for P, A, B
    pVals = [True if x=='P' else False for x in C]
    aVals = [True if x=='A' else False for x in C]
    bVals = [True if x=='B' else False for x in C]
    
    # counter for artistic photographs
    nArtisticPhotos = 0
    
    # now loop through all positions for P, A, B separately
    # skip the position unless it is one of the allowed indices
    # for each valid combination of 3, check for artistic photograph
    for itrP in range(N):
        if not pVals[itrP]: continue
        for itrB in range(N):
            if itrB==itrP: continue
            if not bVals[itrB]: continue
            for itrA in range(N):
                if itrA==itrP or itrA==itrB: continue
                if not aVals[itrA]: continue
                if checkArtisticPhotograph(itrP, itrA, itrB, X, Y):
                    nArtisticPhotos += 1
    
    return nArtisticPhotos

In [38]:
print(getArtisticPhotographCount(5, 'APABA', 1, 2))
print(getArtisticPhotographCount(5, 'APABA', 2, 3))
print(getArtisticPhotographCount(8, '.PBAAP.B', 1, 3))

1
0
3


**Result: 39/39 test cases solved**

### Kaitenzushi
Time limit: 5s

There are $N$ dishes in a row on a kaiten belt, with the $i$th dish being of type $D_i$. Some dishes may be of the same type as one another.

You're very hungry, but you'd also like to keep things interesting. The $N$ dishes will arrive in front of you, one after another in order, and for each one you'll eat it as long as it isn't the same type as any of the previous $K$ dishes you've eaten. You eat very fast, so you can consume a dish before the next one gets to you. Any dishes you choose not to eat as they pass will be eaten by others.

Determine how many dishes you'll end up eating. 

*Please take care to write a solution which runs within the time limit.*

**Constraints**

$1 \leq N \leq 500{,}000 \\
1 \leq K \leq N \\
1 \leq D_i \leq 1{,}000{,}000$

**Sample test cases**
1. $N = 6 \\
D = [1, 2, 3, 3, 2, 1] \\
K = 1$

&emsp;&emsp;Expected return value = 5

2. $N = 6 \\
D = [1, 2, 3, 3, 2, 1] \\
K = 2$

&emsp;&emsp;Expected return value = 4

3. $N = 7 \\
D = [1, 2, 1, 2, 1, 2, 1] \\
K = 2$

&emsp;&emsp;Expected return value = 2

#### Solution

For this problem, I will loop through the $N$ elements of $D$ once, comparing $D_i$ to the last $K$ values in $E$, where $E$ will be a list that I use to keep track of all the dishes I eat. I'll only ever care about the last $K$ elements of $E$. If $D_i$ is not in that slice, I will eat it and I will increment my number of dishes eaten.

Note, for $i$ less than $K$, there will be at most $i$ dishes eaten in $E$, so I will only have to consider $i$ elements in that list, not $K$. But since I plan on using list slices, this is handled fine by python.

**Update: Correct solution below!**

In [64]:
def getMaximumEatenDishCount(N: int, D: List[int], K: int) -> int:
    # counter for dishes eaten
    nDishesEaten = 0
    # list of dishes eaten
    E = []
    
    # loop through D
    for i in range(N):
        # for each dish, compare to the previous K eaten
        # if current dish is in those K, skip
        # otherwise eat and increment counter
        if D[i] in E[-K:]:
            continue
        E.append(D[i])
        nDishesEaten += 1
    return nDishesEaten

In [58]:
print(getMaximumEatenDishCount(6, [1,2,3,3,2,1], 1))
print(getMaximumEatenDishCount(6, [1,2,3,3,2,1], 2))
print(getMaximumEatenDishCount(7, [1,2,1,2,1,2,1], 2))

5
4
2


**Result: 32/33 test cases solved**

**Update**: Above code failed on 1 test case due to exceeding the 5s time limit. I'll try to reproduce below. I can assume that it was in an extreme case with large $N$, $K$, many different values in $D$, large $E$, etc.

In [75]:
import time

In [ ]:
# add some time information to see how the code progresses
def getMaximumEatenDishCount(N: int, D: List[int], K: int) -> int:
    # counter for dishes eaten
    nDishesEaten = 0
    # list of dishes eaten
    E = []
    t1000 = time.time()
    # loop through D
    for i in range(N):
        # for each dish, compare to the previous K eaten
        # if current dish is in those K, skip
        # otherwise eat and increment counter
        if(i%1000==0):
            print(i, time.time() - t1000)
            t1000 = time.time()
        if D[i] in E[-K:]:
            continue
        E.append(D[i])
        nDishesEaten += 1
    return nDishesEaten

In [65]:
N = 500000
K = 500000
D = [i for i in range(N)]
start = time.time()
print(getMaximumEatenDishCount(N, D, K))
end = time.time()
print(end - start)

0 3.814697265625e-06
1000 0.007246732711791992
2000 0.024914026260375977
3000 0.03595113754272461
4000 0.04880118370056152
5000 0.06188702583312988
6000 0.07265615463256836
7000 0.09678506851196289
8000 0.10460400581359863
9000 0.13654208183288574
10000 0.1452488899230957
11000 0.14187884330749512
12000 0.15868091583251953
13000 0.3230760097503662
14000 0.22684907913208008
15000 0.2133798599243164
16000 0.20519804954528809
17000 0.21695995330810547
18000 0.2428739070892334
19000 0.24262714385986328
20000 0.2614750862121582
21000 0.3129079341888428
22000 0.2835381031036377
23000 0.3013150691986084
24000 0.3232438564300537
25000 0.3170340061187744
26000 0.3322021961212158
27000 0.3458528518676758
28000 0.3589010238647461
29000 0.3715629577636719
30000 0.38718485832214355
31000 0.39769577980041504
32000 0.41466307640075684
33000 0.5017738342285156
34000 0.45238399505615234
35000 0.47502684593200684
36000 0.45968103408813477
37000 0.47169041633605957
38000 0.5278670787811279
39000 0.525798

KeyboardInterrupt: 

As shown above, once I get into lists with 10s of thousands of entries, the code becomes very slow and is taking over 1 second to check 1000 dishes, so I would easily hit the 5 second time limit. I need to find a way to speed this up.

I believe that looking for $D_i$ in $E$ is too slow. Perhaps performance will improve using a Set rather than a list. The problem is that since I am keeping track of the last $K$ items eaten, I need some sense of order so I can keep track of the most recent $K$. To this end, I think `OrderedDict` is what I want. There's a sense of order, so I can remove the first item, to keep only the $K$ most recent dishes eaten. But it should still have fast lookup for whether a $D_i$ is a key. The values will all be `None`.

In [93]:
from collections import OrderedDict

# add some time information to see how the code progresses
def getMaximumEatenDishCount(N: int, D: List[int], K: int) -> int:
    # counter for dishes eaten
    nDishesEaten = 0
    # list of dishes eaten as OrderedDict
    E = OrderedDict()
    t1000 = time.time()
    # loop through D
    for i in range(N):
        # for each dish, compare to the previous K eaten
        # which are stored as keys in OrderedDict E
        # if current dish is in those K, skip
        # otherwise eat and increment counter
        # if(i%1000==0):
        #    print(i, time.time() - t1000)
        #    t1000 = time.time()
        if D[i] in E:
            continue
        # dishes eaten stored as keys in E
        E[D[i]] = None
        if len(E) > K:
            E.popitem(last=False)
        nDishesEaten += 1
    return nDishesEaten

In [94]:
print(getMaximumEatenDishCount(6, [1,2,3,3,2,1], 1))
print(getMaximumEatenDishCount(6, [1,2,3,3,2,1], 2))
print(getMaximumEatenDishCount(7, [1,2,1,2,1,2,1], 2))

5
4
2


In [95]:
N = 500000
K = 500000
D = [i for i in range(N)]
start = time.time()
print(getMaximumEatenDishCount(N, D, K))
end = time.time()
print(end - start)

500000
0.197174072265625


Much better performance now!

**Result: 33/33 test cases solved**

### Rotary Lock (Chapter 1)
Time limit: 5s

*Note: Chapter 2 is a harder version of this puzzle.*

You're trying to open a lock. The lock comes with a wheel which has the integers from $1$ to $N$ arranged in a circle in order around it (with integers $1$ and $N$ adjacent to one another). The wheel is initially pointing at 1.

It takes $1$ second to rotate the wheel by $1$ unit to an adjacent integer in either direction, and it takes no time to select an integer once the wheel is pointing at it.

The lock will open if you enter a certain code. The code consists of a sequence of $M$ integers, the $i$th of which is $C_i$. Determine the minimum number of seconds required to select all M of the code's integers in order.

*Please take care to write a solution which runs within the time limit.*

**Constraints**

$ 3 \leq N \leq 50{,}000{,}000 \\
1 \leq M \leq 1{,}000 \\
1 \leq C_{i} \leq N$

**Sample test cases**
1. $N = 3 \\
M = 3 \\
C = [1, 2, 3]$

&emsp;&emsp;Expected return value = 2

2. $N = 10 \\
M = 4 \\
C = [9, 4, 4, 8]$

&emsp;&emsp;Expected return value = 11

#### Solution

For this exercise, I need to calculate the number of rotations needed to get from one number to the next, either rotating forwards or backwards. This amounts to calculating differences modulo $N$.

First, I'll work out the distance from $C_i$ to $C_{i+1}$ depending on the situation:
1. $C_i$ < $C_{i+1}$
    1. Forward: $C_{i+1} - C_{i}$
    2. Backward:
        1. $C_{i}$ to $1$: $C_{i} - 1$
        2. $1$ to $C_{i+1}$: $N - C_{i+1} + 1$
        3. Total: $C_i - C_{i+1} + N$
2. $C_i$ > $C_{i+1}$
    1. Forward: 
        1. $C_i$ to $N$: $N - C_i$
        2. $N$ to $C_{i+1}$: $C_{i+1}$
        3. Total: $C_{i+1} - C_{i} + N$
    2. Backward: $C_i - C_{i+1}$

Take for example rotating from $1$ to $9$ on an $N = 10$ lock. If I go forward it takes $9 - 1 = 8$ seconds. If I go backward, it takes $0$ seconds to get to $1$, then $10 - 9 + 1 = 2$ seconds to get to $9$ for a total of $2$ seconds.

Alternatively, if we try to get from $9$ to $8$ on the same $N = 10$ lock, it would take $1 + 8 = 9$ seconds going forward or $9 - 8 = 1$ second going backward.

Finally, if we try to get from $4$ to $9$, it takes $5$ seconds going forward and backward. The point of equality is when the distance between the numbers is $N / 2$, then it doesn't matter if I go forward or backwards, it's exactly on the opposite side of the circle. For odd $N$, one direction will always be faster.

Now, I return to these cases and examine them using modular arithmetic:
1. $1$ to $9$ on $N = 10$
    1. $(9 - 1) \bmod 10 = 8$
    2. $(1 - 9) \bmod 10 = 2$
2. $9$ to $8$ on $N = 10$
    1. $(8 - 9) \bmod 10 = 9$
    2. $(9 - 8) \bmod 10 = 1$
3. $4$ to $9$ on $N = 10$
    1. $(9 - 4) \bmod 10 = 5$
    2. $(4 - 9) \bmod 10 = 5$

So, the fastest way between $C_{i}$ and $C_{i+1}$ is $$\min\big((C_i - C_{i+1})\bmod N, (C_{i+1} - C_i) \bmod N\big)$$.

I will attempt to answer the puzzle by computing both expressions and choosing the minimum. I will do this using numpy arrays

In [90]:
from typing import List
# Write any import statements here
import numpy as np

def getMinCodeEntryTime(N: int, M: int, C: List[int]) -> int:
    # Write your code here
    
    # calculate diffs c[i+1] - c[i]
    # and c[i] - c[i+1] (just flip the sign)
    # and take mod N
    # need to prepend 1 since that's where we start
    c1 = np.diff(C, prepend=1)
    c2 = -c1
    c1 = c1 % N
    c2 = c2 % N
    
    # get minimum distance, element-wise
    # then sum all entries to get total time
    c3 = np.minimum(c1, c2)
    return np.sum(c3)

In [91]:
print(getMinCodeEntryTime(3,3,[1,2,3]))
print(getMinCodeEntryTime(10,4,[9,4,4,8]))

2
11


**Result: 32/32 test cases solved**

### Scoreboard Inference (Chapter 1)
Time limit: 5s

*Note: Chapter 2 is a harder version of this puzzle. The only difference is a larger set of possible problem point values.*

You are spectating a programming contest with $N$ competitors, each trying to independently solve the same set of programming problems. Each problem has a point value, which is **either 1 or 2**.

On the scoreboard, you observe that the $i$th competitor has attained a score of $S_i$, which is a positive integer equal to the sum of the point values of all the problems they have solved.

The scoreboard does not display the number of problems in the contest, nor their point values. Using the information available, you would like to determine the minimum possible number of problems in the contest.

**Constraints**

$ 1 \leq N \leq 500{,}000 \\
1 \leq S_i \leq 1{,}000{,}000{,}000$

**Sample test cases**
1. $N = 6 \\
S = [1, 2, 3, 4, 5, 6]$

&emsp;&emsp;Expected return value = 4

2. $N = 4 \\
S = [4, 3, 3, 4]$

&emsp;&emsp;Expected return value = 3

3. $N = 4 \\
S = [2, 4, 6, 8]$

&emsp;&emsp;Expected return value = 4

#### Solution

The first thing I notice is that the upper bound on $S_i$ being $10^9$ means it's not feasible to do too much explicitly trying to make combinations of point allocations. Sample test case 3 is illustrative because it shows how dependent the problem is on factors of $2$.

When all $S_i$ are even, i.e. $2m_{i}$ for some $m_i$, then the minimum number of problems is just given by the largest $m_i$. In sample test case 3, this is $4$. When there are odd numbers, there must be at least $1$ problem worth $1$ problem.

Now, consider test case 1. Since there is one contestant with $6$ points, we can start with having 3 problems worth $2$ points. Then that player may have gotten all of those. This also gives us all even numbers below $6$, so we have handled $2$ and $4$. Finally, since there are players with odd values, they must have scored a $1$ point problem, so if we add one $1$ point problem we can make all odd numbers up to $7$. This gives us $4$ total problems needed.

Rather than starting with 3 problems worth $2$ points we might have started with just 2. This gets us to $2$ or $4$. Adding a $1$ point problem gets us to $1$, $3$, $5$. To get to $6$ we just need one more $1$ point problem. In this scenario we need $4$ total problems as well.

It doesn't matter whether we choose $m_{i}$ problems worth $2$ points plus $1$ problem worth $1$ point, or $m_{i} - 1$ problems worth $2$ points plus $2$ problems worth $1$ point. Either way, we need $m_{i} + 1$ problems to make all possible scores up to $2*m_i$.

So this problem becomes solving for $m_i$.

Note that the previous work assumed that the largest $S_i$ was even. If it is odd, then $S_i - 1 = 2*m_j$ for some $m_j$. Then $m_j$ problems worth $2$ points gives us all even values up to $2*m_{j}$ and one $1$ point problem gives us all odd numbers up to $2*m_j + 1 = S_i$. The total problems needed is $m_j + 1$ then.

So in both cases (max $S_i$ is even or odd) when there is at least 1 odd number in $S$, we need $m_j + 1$ where:
$$ m_j = \begin{cases}
S_{\max} / 2 & S_{max} \text{ even} \\
(S_{\max} - 1) / 2 & S_{max} \text{ odd}
\end{cases} $$
where $S_{\max} = \max_{1 \leq i \leq N} S_i $.

If all elements $S_i$ are even then it's just $m_j$.

All we need now is just to know whether $S$ is entirely even.

In [94]:
from typing import List
# Write any import statements here

def getMinProblemCount(N: int, S: List[int]) -> int:
    # get max in S
    # m_j defined above can be calculated with integer division
    m = max(S) // 2
    
    # loop through elements, checking until I find an odd
    for itr in S:
        if itr % 2 == 1:
            return m + 1
    # all elements are even
    return m

In [95]:
print(getMinProblemCount(6, [1,2,3,4,5,6]))
print(getMinProblemCount(4, [4,3,3,4]))
print(getMinProblemCount(4, [2,4,6,8]))

4
3
4


**Result: 34/34 test cases solved**

### Stack Stabilization (Chapter 1)
Time limit: 5s

*Note: Chapter 2 is a harder version of this puzzle.*

There's a stack of $N$ inflatable discs, with the $i$th disc from the top having an initial radius of $R_i$ inches.

The stack is considered *unstable* if it includes at least one disc whose radius is larger than or equal to that of the disc directly under it. In other words, for the stack to be stable, each disc must have a strictly smaller radius than that of the disc directly under it.

As long as the stack is unstable, you can repeatedly choose any disc of your choice and deflate it down to have a radius of your choice which is strictly smaller than the disc’s prior radius. The new radius must be a positive integer number of inches.

Determine the minimum number of discs which need to be deflated in order to make the stack stable, if this is possible at all. If it is impossible to stabilize the stack, return $−1$ instead.

**Constraints**

$ 1 \leq N \leq 50 \\
1 \leq R_i \leq 1{,}000{,}000{,}000$

**Sample test cases**
1. $N = 5 \\
R = [2, 5, 3, 6, 5]$

&emsp;&emsp;Expected return value = 3

2. $N = 3 \\
R = [100, 100, 100]$

&emsp;&emsp;Expected return value = 2

3. $N = 4 \\
R = [6, 5, 4, 3]$

&emsp;&emsp;Expected return value = -1

#### Solution

The stack cannot be made stable if $R[-1] < N$ (since all discs are positive integer radii and there are $N$ discs, if they are stacked in order each increasing by the minimum amount of $1$, then they will range from $1$ to $N$, and $R[-1]$ needs to be largest). If I start at the end of $R$, I can check if $R[-1] < N$. Then I can check if $R[-2] < R[-1]$. If not $R[-2]$ needs to be deflated. Since discs can only be deflated, all discs need to be smaller than $R[-1]$.

Repeating the same argument as above, $R[-2]$ needs to be $\geq N - 1$. If not, then it cannot be made stable. $R[-3]$ should be smaller or deflated to be smaller than $R[-2]$ and at least $N - 2$. This goes on all the way until $R[0] = R[-N]$ which must be the smallest disc. If the stack can be iteratively made stable all the way from the bottom to top, then it is fully stable.

In [13]:
from typing import List
# Write any import statements here

def getMinimumDeflatedDiscCount(N: int, R: List[int]) -> int:
  # Write your code here
    # counter for necessary deflations
    nDeflated = 0
    
    # loop through discs from bottom to top, 1 to N
    for i in range(1,N+1):
        
        # for all discs (other than the last one which has to remain as is)
        # if it is bigger or equal to disc below it, deflate it
        # leave it as large as possible (disc below it minus 1)
        # to make rest of process easier
        if i > 1:
            if R[-i] >= R[-i+1]:
                R[-i] = R[-i+1] - 1
                nDeflated += 1
                
        # discs need to meet minimum size for their position
        # for -1 min is N, -2 min is N - 1, for -i min is N - (i - 1)
        # if discs are smaller than this, stack can't be stable so return -1
        if R[-i] < (N - (i - 1)):
            return -1
    
    # done looping and never ran into disc that didn't meet stability requirements
    return nDeflated

In [15]:
print(getMinimumDeflatedDiscCount(5, [2,5,3,6,5]))
print(getMinimumDeflatedDiscCount(3, [100,100,100]))
print(getMinimumDeflatedDiscCount(4, [6,5,4,3]))

3
2
-1


**Result: 33/33 test cases solved**

### Uniform Integers
Time limit: 5s

A positive integer is considered *uniform* if all of its digits are equal. For example, $222$ is uniform, while $223$ is not.
Given two positive integers $A$ and $B$, determine the number of uniform integers between $A$ and $B$, inclusive.

*Please take care to write a solution which runs within the time limit.*

**Constraints**

$1 \leq A \leq B \leq 10^{12}$

**Sample test cases**
1. $A = 75 \\
B = 300$

&emsp;&emsp;Expected return value = 5

2. $A = 1 \\
B = 9$

&emsp;&emsp;Expected return value = 9

3. $A = 999999999999 \\
B = 999999999999$

&emsp;&emsp;Expected return value = 1

#### Solution

Given the large upper bound on $A$ and $B$ and the required return, I think this problem will come down to calculating orders of magnitude between the two integers. For example, the $10^0$ order of magnitude has $9$ total uniform integers, each of the $9$ digits. Similarly, the $10^1$ order of magnitude similarly has $9$ uniform integers: $11, 22, 33, \dots, 99$. For each entire order of magnitude spanned between $A$ and $B$, we will count $9$ uniform integers. 

Next, we need to consider the partial orders of magnitude. Looking at sample test case 1, since $A = 75$ is less than $77$, we pick up $3$ uniform integers in the $10^1$ order of magnitude ($77, 88, 99$). Then, since $300$ is less than $333$ we only pick up $2$ unfirorm integers from the $10^2$ order ($111, 222$). For a total of $5$.

We can use integer division to get both the order of magnitude (how many times we divide by 10 before getting a single digit) and the digit in that order of magnitude (the digit left over after all divisions by 10). Then we can compare to the uniform integer at that order of magnitude with that starting digit. This will tell us how many uniform integers to count for the partial orders of magnitude.

Finally, we have to consider edge cases when $A$ and $B$ are themselves uniform integers or equal to one another to make sure we account for those correctly.

In [33]:
# helper function to get order of magnitude and leading digit
# repeated integer division by 10 will remove rightmost digit
# until we are left with just the leftmost digit
# and the number of divisions necessary gives us the order of magnitude
def getOrderOfMagnitudeAndDigit(N: int) -> (int, int):
    orderOfMagnitude = 0
    digit = N
    while digit >= 10:
        digit = digit // 10
        orderOfMagnitude += 1
    return orderOfMagnitude, digit

In [34]:
print(getOrderOfMagnitudeAndDigit(782))
print(getOrderOfMagnitudeAndDigit(78234500))
print(getOrderOfMagnitudeAndDigit(7))
print(getOrderOfMagnitudeAndDigit(100))

(2, 7)
(7, 7)
(0, 7)
(2, 1)


In [35]:
# helper function to get uniform integer
# for a given order of magnitude m and digit n
# answer is integer with (m+1) digits equal to n
def getUniformInteger(m: int, n: int) -> int:
    ans = 0
    for i in range(m+1):
        ans += n*pow(10,i)
    return ans

In [37]:
print(getUniformInteger(2,7))
print(getUniformInteger(7,7))
print(getUniformInteger(0,7))
print(getUniformInteger(2,1))

777
77777777
7
111


In [71]:
def getUniformIntegerCountInInterval(A: int, B: int) -> int:
    # counter to be returned as answer
    nUniformIntegers = 0
    
    # replace A, B with X, Y with
    # X < Y always to make things easier
    X = min(A,B)
    Y = max(A,B)
    
    # get order of magnitude, leading digit for A and B
    # get uniform digit at those magnitudes with those digits
    # to compare for partial orders of magnitude
    mX, nX = getOrderOfMagnitudeAndDigit(X)
    mY, nY = getOrderOfMagnitudeAndDigit(Y)
    xx = getUniformInteger(mX, nX)
    yy = getUniformInteger(mY, nY)
    
    # each *complete* order of magnitude between numbers 
    # (happens when mY > mX + 1)
    # gives 9 uniform integers
    if mY > mX + 1:
        nUniformIntegers += 9*(mY - mX - 1)
        
    # partial orders of magnitude depend whether mX = mY
    # if mX = mY just that order of magnitude matters
    # otherwise there is some partial orders of magnitude for mX and mY
    if mX == mY:
        # only want to count uniform integers between X, Y
        # 9 total - those less than X - those greater than Y
        # if X > xx, then all nX are less than X, otherwise (nX - 1)
        # if Y < yy, then (9 - nY + 1) are greater than Y, otherwise (9 - nY)
        tmp = 9
        if X > xx:
            tmp -= nX
        else:
            tmp -= (nX - 1)
        if Y < yy:
            tmp -= (9 - nY + 1)
        else:
            tmp -= (9 - nY)
        nUniformIntegers += tmp
            
    else:
        # for X, add uniform integers greater than X with mX
        # for Y, add uniform integers less than Y with mY
        if X > xx:
            nUniformIntegers += (9 - nX)
        else:
            nUniformIntegers += (9 - nX + 1)
        if Y < yy:
            nUniformIntegers += nY - 1
        else:
            nUniformIntegers += nY
            
    return nUniformIntegers

In [74]:
print(getUniformIntegerCountInInterval(75,300))
print(getUniformIntegerCountInInterval(1,9))
print(getUniformIntegerCountInInterval(999999999999,999999999999))
print(getUniformIntegerCountInInterval(77,300))
print(getUniformIntegerCountInInterval(77,333))
print(getUniformIntegerCountInInterval(76,333))

5
9
1
5
6
6


**Result: 33/33 test cases solved**